## reading a pdf file
code from [pdfchat](https://github.com/bhaskatripathi/pdfGPT/blob/main/api.py)

In [5]:
import re
import fitz

def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text


def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count
    
    if end_page is None:
        end_page = total_pages
        
    text_list = []
    
    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)
        
    doc.close()
    return text_list

In [6]:
pdf_file = "StarCoder.pdf"
text_list = pdf_to_text(pdf_file)